In [1]:
#| default_exp utils

In [13]:
#| export
from fasthtml.common import *
import frontmatter
# import markdown
from pathlib import Path
from datetime import datetime

In [26]:
#| export
POSTS_DIR = Path("../posts")

In [15]:
#| export
def load_post_by_slug(slug: str) -> Optional[Dict]:
    """Load a single post by its slug."""
    # Try to find the post file matching the slug
    for post_file in POSTS_DIR.glob("*.md"):
        if slug in post_file.stem: post = frontmatter.load(post_file)
        return {
            'slug': slug,
            'title': post.get('title', 'Untitled'),
            'date': post.get('date', datetime.now()),
            'tags': post.get('tags', []),
            'excerpt': post.get('excerpt', ''),
            'content': post.content,
        }

    return None

In [24]:
#| export
def load_all_posts() -> List[Dict]:
    """Load all blog posts from the posts directory."""
    if not POSTS_DIR.exists(): return []
    posts = []
    for post_file in POSTS_DIR.glob("*.md"):
        post = frontmatter.load(post_file)
        slug = extract_slug_from_filename(post_file.name)
        posts.append({
            'slug': slug,
            'title': post.get('title', 'Untitled'),
            'date': post.get('date', datetime.now()),
            'tags': post.get('tags', []),
            'excerpt': post.get('excerpt', '')
        })

    return posts

In [17]:
#| export
def get_all_tags() -> List[str]:
    """Get sorted list of all unique tags across all posts"""
    tags = set()
    posts = load_all_posts()
    for post in posts: tags.update(post.get('tags', []))
    return sorted(list(tags))

In [18]:
#| export
def filter_posts(posts: List[Dict], selected_cats: List[str]) -> List[Dict]:
    """Filter posts by selected categories usin AND logic"""
    if not selected_cats: return posts
    return [p for p in posts if all(cat in p.get('tags', []) for cat in selected_cats)]

In [19]:
#| export
def get_enabled_tags(all_posts: List[Dict], selected_cats: List[str], all_tags: List[str]) -> List[str]:
    """Calculate which tags should be enabled based on current selection"""
    if not selected_cats:
        # No filters: enable all tags that appear in at least one post
        return [tag for tag in all_tags if any(tag in p.get('tags', []) for p in all_posts)]

    enabled_tags = set()
    for tag in all_tags:
        # Already selected tags are always enabled (for removal)
        if tag in selected_cats:
            enabled_tags.add(tag)
            continue
        
        # Check if adding this tag would still return posts
        test_cats = selected_cats + [tag]
        if filter_posts(all_posts, test_cats): enabled_tags.add(tag)
            
    return list(enabled_tags)

In [20]:
# #| export
# def markdown_to_html(content: str) -> str:
#     """Convert markdown content to HTML with extensions."""

#     md = markdown.Markdown(extensions=[
#         'fenced_code',      # For code blocks
#         'codehilite',       # Code syntax highlighting
#         'tables',           # Table support
#         'toc',              # Table of contents
#         'nl2br',            # Newline to <br>
#     ])

#     return md.convert(content)

In [21]:
#| export
def extract_slug_from_filename(filename: str) -> str:
    """Extract slug from filename (format: YYYY-MM-DD-slug-name.md)"""
    stem = Path(filename).stem
    parts = stem.split('-')
    # Skip first 3 parts (year, month, day)
    return '-'.join(parts[3:]) if len(parts) > 3 else stem

In [27]:
from nbdev.export import nb_export
nb_export('02_utils.ipynb', '../')